In [1]:
import os
from langchain_openai import OpenAI
from langchain.memory.buffer import ConversationBufferMemory
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.chains.llm import LLMChain
import streamlit as st
from dotenv import load_dotenv, find_dotenv

from constants import FEW_SHOT_EXAMPLE
import warnings
warnings.filterwarnings("ignore")

In [2]:
load_dotenv(find_dotenv())

# load the API Key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "")

In [3]:
# intialize the model
llm_model = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0.3,
    max_tokens=1500
)

# intialize the Memory
memory = ConversationBufferMemory()

# Intialize the Prompt example
example_travel_prompt = PromptTemplate(
    input_variables=["travel_query", "travel_response"],
    template="""
        You are a intelligent Travel Agent Advisor which helps to plan a trip for a customer from one destination to another.
        Now based on the source and destination for which a person asked to plan a trip for him, you have to provide your response
        in accurate, precise and point wise manner so that a customer easily understands the mode of transportation,
        fare for each mode of transportation, flight number, cab number, buses number and suggest the alternate route/mode of transportation
        if not present directly from source and destination. Your job is also not provide any incorrect or not feasible information which is
        not important to customer. Analysis the request first step by step fully and then provide your response.
        {travel_query} \n {travel_response}
"""
)

In [4]:
FEW_SHOT_EXAMPLE

[{'travel_query': 'Can you plan a trip from Shahdara, Delhi to London?',
  'travel_response': "\n        Since this is a international travel request, let's understand the process of travelling from one country to another.\n        First we have to book a flight which might have some layover in between and might be a direct flight.To book a flight you can \n        use any of the online service provider. \n        \n        The layover options are: Gulf Air flight number GF 131, layover of 3hrs 25 mins at Bahrain and then to destination London-Heathrow Apt.\n        Total  cost will be Rs. 28343 per person.\n\n        The direct options are: Vistra flight UK 17 has total cost Rs. 67225, British Airways BA 142 has total cost Rs. 69891\n\n        Now, to reach airport to pick the flight, we need to either travel via Cab/Metro/Train/Buses. The nearest Airport for delhi is Indira \n        Gandhi International Airport (IGI)\n        Now there are no direct Train which goes to Delhi's airpo

In [5]:
print(example_travel_prompt.format(**FEW_SHOT_EXAMPLE[0]))


        You are a intelligent Travel Agent Advisor which helps to plan a trip for a customer from one destination to another.
        Now based on the source and destination for which a person asked to plan a trip for him, you have to provide your response
        in accurate, precise and point wise manner so that a customer easily understands the mode of transportation,
        fare for each mode of transportation, flight number, cab number, buses number and suggest the alternate route/mode of transportation
        if not present directly from source and destination. Your job is also not provide any incorrect or not feasible information which is
        not important to customer. Analysis the request first step by step fully and then provide your response.
        Can you plan a trip from Shahdara, Delhi to London? 
 
        Since this is a international travel request, let's understand the process of travelling from one country to another.
        First we have to book a flight wh

In [6]:
# Intialize few short template with examples
few_shot_prompt = FewShotPromptTemplate(
    examples=FEW_SHOT_EXAMPLE,
    example_prompt=example_travel_prompt,
    suffix="Travel Query: {travel_query}",
    input_variables=["travel_query", "travel_response"]
)

In [9]:
# Created a LLM chain to run the model
travel_chain = LLMChain(
    llm=llm_model,
    memory=memory,
    prompt=few_shot_prompt
)

In [10]:
print(travel_chain.invoke({"travel_query":"Plan a trip from Hauz Khas, Delhi to Goa?"})["text"])



Since this is a domestic travel request, let's understand the process of travelling from one city to another city. 
First we have to book the mode of transport which we need to travel from source to destination. It might be flight/train/cab/buses.

There are no direct flights from Hauz Khas to Goa. 
IndiGo flight number 6E 744 will first fly from Delhi to Mumbai which has layover of 1hr 20 mins and then from Mumbai to Goa.
The nearest airport from Hauz Khas is DEL, Delhi and nearest airport from Goa is GOI. The total cost will be Rs. 9000.

To reach DEL airport you can make use of Cabs which will cost around Rs. 500. You can also take a metro from Hauz Khas Metro Station to
New Delhi Metro Station and then take Airport Express Line to reach DEL airport. The total cost will be Rs. 60.

For Train, you can book 22633 - Nizamuddin Express which starts from Hauz Khas at 05:50 PM and reach to Madgaon Station at 07:20 AM.
The journey has 13 stops and covers 1875 KM. Total cost will be betwe

In [11]:
print(travel_chain.invoke({"travel_query":"Plan a trip from New York to bangalore?"})["text"])



This is an international travel request.
First we have to book a flight which might have some layover in between and might be a direct flight. 

Emirates flight number Ek-204 from John F Kennedy Airport, Terminal-4, stops at Dubai International Airport for a layover of 3hrs 55 mins 
and then to Kempegowda International Airport, Bengaluru. Total cost Rs. 61869.

Air India flight number AI-102 departs from John F Kennedy Airport, Terminal-4 and arrives directly at Kempegowda International Airport.
Total cost will be Rs. 64197.

Nearest Airport from New York is John F Kennedy Airport. To reach there, we either need taxi/train/cab/buses/autos.
Cabs can be booked from Uber/Ola having an average fare of $60. Autos will have $7. 

Nearest Metro station is Jamaica Station which is 23 mins drive away. Cost would be $2.75.

To travel from New York to John F Kennedy Airport, you can take a train from Penn Station to Jamaica Station, which takes approximately 
20 minutes and operates every 10 mi

In [13]:
print(travel_chain.invoke({"travel_query": "Plan a trip from Shivaji park, Delhi to Saket, Delhi?"})["text"])



Since this is a domestic travel request, let's understand the process of travelling from one city to another city. 
First we have to book the mode of transport which we need to travel from source to destination. It might be flight/train/cab/buses.

For this request, the most convenient and economical option is to travel by Metro. The nearest metro station to Shivaji Park is Rajiv Chowk 
which is 4.5 km away. From Rajiv Chowk, you can take the Yellow Line towards Huda City Centre and get down at Saket Metro Station. 
The total cost will be Rs. 30 and the journey will take approximately 30 minutes.

If you prefer to travel by cab, you can book a cab from Uber/Ola or any other private taxi service. The fare will be around Rs. 150-200 
and the journey will take approximately 45 minutes depending on the traffic.

For buses, you can make use of Delhi Transport Corporation (DTC) buses which run within Delhi NCR. The nearest bus stop to Shivaji Park is 
Shivaji Stadium Bus Stop. From there, 

In [18]:
print(travel_chain.invoke({"travel_query":"Plan a trip from panchkula, harayana to Kashmir?"})["text"])



Since this is a domestic travel request, let's understand the process of travelling from one city to another city. 
First we have to book the mode of transport which we need to travel from source to destination. It might be flight/train/cab/buses.

For this request, the best mode of transportation would be a flight. The nearest airport to Panchkula, Haryana is Chandigarh Airport (IXC)
and the nearest airport to Kashmir is Srinagar Airport (SXR). There are no direct flights available from Chandigarh Airport to Srinagar Airport.
However, there are connecting flights with layovers.

The options for connecting flights are:

1. Air India flight AI 9836 from Chandigarh to Delhi, with a layover of 2 hours and then connecting flight AI 825 from Delhi to Srinagar.
Total cost would be around Rs. 11,000 to Rs. 15,000.

2. IndiGo flight 6E 251 from Chandigarh to Delhi, with a layover of 2 hours and then connecting flight 6E 555 from Delhi to Srinagar.
Total cost would be around Rs. 10,000 to Rs.

In [16]:
print(travel_chain.invoke({"travel_query": "Trip from Jamshedpur, Jharkhand to Rishikesh, Uttarkand?"})["text"])



Since this is a domestic travel request, let's understand the process of travelling from one city to another city. 
First we have to book the mode of transport which we need to travel from source to destination. It might be flight/train/cab/buses.

There are no direct flights available from Jamshedpur to Rishikesh. The nearest airport from Jamshedpur is Sonari Airport (IXW) and 
the nearest airport from Rishikesh is Jolly Grant Airport (DED).

To reach Sonari Airport, you can take a cab which will cost around Rs. 600 to Rs. 800. Alternatively, you can also take a bus from 
Jamshedpur to Ranchi and then from Ranchi to Dehradun. From Dehradun, you can take a cab to reach Rishikesh.

For train, you can book a ticket on the Howrah-Dehradun Express (13009) which will take you from Tatanagar Junction (TATA) to 
Dehradun Railway Station (DDN). The journey takes approximately 24 hours and the cost ranges from Rs. 500 to Rs. 1500.

For buses, you can take a bus from Jamshedpur to Ranchi and t

In [23]:
print(travel_chain.invoke({"travel_query": "Trip from Manali to Vietnam?"})["text"])



Since this is an international travel request, let's understand the process of travelling from one country to another.
First we have to book a flight which might have some layover in between and might be a direct flight. 

To book a flight you can use any of the online service provider. 
The layover options are: Air India flight number AI-981, layover of 2hrs 40 mins at New Delhi and then to destination Hanoi, Vietnam.
Total cost will be Rs. 27,000 per person.

The direct options are: Vietnam Airlines flight VN-194 has total cost Rs. 40,000, Jet Airways flight 9W-690 has total cost Rs. 42,000.

Now, to reach airport to pick the flight, we need to either travel via Cab/Bus. The nearest Airport for Manali is Bhuntar Airport.
There are no direct buses from Manali to Bhuntar Airport, so we have to take a cab which will cost around Rs. 1000 to Rs. 1500.

The nearest bus station to Manali is Manali Bus Stand. From there, you can take a bus to Kullu Bus Stand and then from Kullu Bus Stand
t